# **IMPORT LIBS**

In [1]:
import numpy as np
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.recommenders import CML
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler
import numpy as np
import pandas as pd
import os
import pickle
import random


# **GET THE DATASET**

In [2]:
seed = 2384795
np.random.seed(seed=seed)

folder_name = f"./Dataset/"

if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

## Load the dataset

In [3]:
file_path = 'yahoo_ymusic_v1/ydata-ymusic-rating-study-v1_0-train.txt'

# Load the training set into a DataFrame
df_train = pd.read_csv(folder_name+file_path, sep='\t',names=["UserID","SongID","Rating"], header=None)
df_train.head(10)

,UserID,SongID,Rating
0,1,14,5
1,1,35,1
2,1,46,1
3,1,83,1
4,1,93,1
5,1,94,1
6,1,153,5
7,1,170,4
8,1,184,5
9,1,194,5


## Convert to implicit

"We treat items rated greater than or equal to 4 as relevant, and others as irrelevant, as suggested by prior literature."

In [4]:
POSITIVE_THRESHOLD = 4
df_train['ImplicitRating'] = np.where(df_train['Rating'] >= POSITIVE_THRESHOLD, 1, 0)

df_train.head(10)

,UserID,SongID,Rating,ImplicitRating
0,1,14,5,1
1,1,35,1,0
2,1,46,1,0
3,1,83,1,0
4,1,93,1,0
5,1,94,1,0
6,1,153,5,1
7,1,170,4,1
8,1,184,5,1
9,1,194,5,1


## Check the number of users and items in the training set

"The training set contains 300K ratings given by 15.4K users against 1K songs through natural interactions."

In [5]:
min_user = df_train["UserID"].min()
max_user = df_train["UserID"].max()

min_item = df_train["SongID"].min()
max_item = df_train["SongID"].max()

max_item, max_user

(1000, 15400)

# **GET UNBIASED TESTSET**

## Load the unbiased testset and convert it to implicit

In [6]:
file_path = 'Dataset/yahoo_ymusic_v1/ydata-ymusic-rating-study-v1_0-test.txt'
df_test = pd.read_csv(file_path, sep='\t',names=["UserID","SongID","Rating"], header=None)  # sep='\t' for tab-separated values
df_test['ImplicitRating'] = np.where(df_test['Rating'] >= POSITIVE_THRESHOLD, 1, 0)
df_test.head(10)

,UserID,SongID,Rating,ImplicitRating
0,1,49,1,0
1,1,126,1,0
2,1,138,1,0
3,1,141,1,0
4,1,177,1,0
5,1,268,3,0
6,1,511,1,0
7,1,587,1,0
8,1,772,5,1
9,1,941,1,0


## Check the number of users and items in the training set

"The testing set is collected by asking a subset of 5.4K users to rate 10 randomly selected songs."

In [7]:
df_test["UserID"].max(), df_test["SongID"].max(), int(df_test.shape[0]/df_test["UserID"].max())

(5400, 1000, 10)

## Filter unbiased testset

"We filter the testing set by retaining users who have at least a relevant and an irrelevant song in the testing set and two relevant songs in the training set."

In [8]:
# Select users with at least an irrelevant song in the unbiased testset
usersWithNegativeInteractionInTest = df_test[df_test["ImplicitRating"] == 0]["UserID"].unique()

# Select UserID of users with at least a relevant song in testset
usersWithPositiveInteractionInTest = df_test[df_test["ImplicitRating"] == 1]["UserID"].unique()

# Select UserID of users with at least two relevant song in trainset
usersWithTwoPositiveInteractions = df_train[df_train["ImplicitRating"] == 1].groupby("UserID").filter(lambda x: len(x) >= 2)['UserID'].unique()

# Compute the intersection
set1 = set(usersWithNegativeInteractionInTest)
set2 = set(usersWithPositiveInteractionInTest)
set3 = set(usersWithTwoPositiveInteractions)
valid_users_testset = set1 & set2 & set3

# Filter the testset
df_test_filtered = df_test[df_test["UserID"].isin(valid_users_testset)]

"2296 users satisfy these requirements."

In [9]:
len(valid_users_testset)

2296

## Shape the unbiased test set

In [10]:
# From the dataframe, for each row where ImplicitRating is 1, append [userID, itemID] to unbiased_pos_test_set
# and for each row where ImplicitRating is 0, append [userID, itemID] to unbiased_neg_test_set

unbiased_pos_test_set = df_test_filtered[df_test_filtered["ImplicitRating"] == 1][["UserID", "SongID"]].values
unbiased_neg_test_set = df_test_filtered[df_test_filtered["ImplicitRating"] == 0][["UserID", "SongID"]].values

## Save unbiased test set

In [11]:
unbiased_pos_test_set_df = pd.DataFrame(unbiased_pos_test_set)
unbiased_neg_test_set_df = pd.DataFrame(unbiased_neg_test_set)

unbiased_pos_test_set_df.columns = ["user_id","item_id"]
unbiased_neg_test_set_df.columns = ["user_id","item_id"]

structured_data_pos_test_set_unbiased = unbiased_pos_test_set_df.to_records(index=False)
structured_data_neg_test_set_unbiased = unbiased_neg_test_set_df.to_records(index=False)

np.save(folder_name + "unbiased-test_arr_pos.npy", structured_data_pos_test_set_unbiased)
np.save(folder_name + "unbiased-test_arr_neg.npy", structured_data_neg_test_set_unbiased)

# **GET BIASED TESTSET**

## Load the training testset and convert it to implicit

"We additionally held out a biased testing set (biased-testing) from the training set...".

In [12]:
file_path = 'yahoo_ymusic_v1/ydata-ymusic-rating-study-v1_0-train.txt'

# Load the training set into a DataFrame
df_train = pd.read_csv(folder_name+file_path, sep='\t',names=["UserID","SongID","Rating"], header=None)
df_train['ImplicitRating'] = np.where(df_train['Rating'] >= POSITIVE_THRESHOLD, 1, 0)
df_train = df_train[df_train["UserID"].isin(valid_users_testset)]
df_train.head(10)

,UserID,SongID,Rating
0,1,14,5
1,1,35,1
2,1,46,1
3,1,83,1
4,1,93,1
5,1,94,1
6,1,153,5
7,1,170,4
8,1,184,5
9,1,194,5


## Extract the biased test set and shape it

"...by randomly sampling 300 songs for each user."

In [16]:
# Precompute, for each user, the list of songs with a relevant rating
user_positive_ratings = df_train[df_train["ImplicitRating"] == 1].groupby("UserID")["SongID"].apply(set)

# Initialize the range of indexes for the items
items_ids = np.arange(min_item, max_item + 1)
# Set the number of songs for each user
SONGS_FOR_BIASED_TEST = 300

# HYPOTHESIS MAN

pos_test_set = []
neg_test_set = []

for user_id in valid_users_testset:
    np.random.shuffle(items_ids)
    test_items = set(items_ids[-SONGS_FOR_BIASED_TEST:])
    pos_ids = user_positive_ratings.get(user_id, set()) & test_items

    # Set them to 0 so that they will no longer be used in training set
    df_train.loc[(df_train['SongID'].isin(pos_ids)) & (df_train['UserID'] == user_id), 'ImplicitRating'] = 0

    for id in test_items:
        if id in pos_ids:
            pos_test_set.append([user_id, id])
        else:
            neg_test_set.append([user_id, id])

pos_test_set = np.array(pos_test_set)
neg_test_set = np.array(neg_test_set)

## Save the biased test set

In [17]:
pos_test_set_df = pd.DataFrame(pos_test_set)
neg_test_set_df = pd.DataFrame(neg_test_set)

pos_test_set_df.columns = ["user_id","item_id"]
neg_test_set_df.columns = ["user_id","item_id"]

structured_data_pos_test_set = pos_test_set_df.to_records(index=False)
structured_data_neg_test_set = neg_test_set_df.to_records(index=False)

np.save(folder_name + "biased-test_arr_pos.npy", structured_data_pos_test_set)
np.save(folder_name + "biased-test_arr_neg.npy", structured_data_neg_test_set)

## Take couples user-item filtering out the irrelevant ones

In [18]:
# Only take the couples (user, item) with relevant rating
new_df = df_train[df_train['ImplicitRating'] != 0]
new_df = new_df.drop(columns=['Rating', 'ImplicitRating'])

# Define a dictionary for renaming columns
rename_dict = {
    'UserID': 'user_id',
    'SongID': 'item_id'
}

# Rename the columns
new_df = new_df.rename(columns=rename_dict)

# Convert the DataFrame to a structured array
train_data = new_df.to_records(index=False) 

## Save the training set changes

In [20]:
np.save(folder_name + "training_arr.npy", train_data)

# **TRAIN THE MODEL**

## Model choice

In [21]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')

MODEL_CLASS = CML
MODEL_PREFIX = "cml"
DATASET_NAME = "yahoo"
OUTPUT_FOLDER = "./Output/"
OUTPUT_PATH = OUTPUT_FOLDER + MODEL_PREFIX + "-" + DATASET_NAME + "/"
OUTPUT_PREFIX = str(OUTPUT_PATH) + str(MODEL_PREFIX) + "-" + str(DATASET_NAME)

if os.path.exists(OUTPUT_PATH) == False:
    os.makedirs(OUTPUT_PATH)


## Train

In [ ]:
# Avoid tensorflow using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()


model = MODEL_CLASS(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(), 
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=model, sampler=sampler,
                                     eval_save_prefix=OUTPUT_PATH + DATASET_NAME,
                                     item_serving_size=500)
auc_evaluator = AUC()
model_trainer.train(num_itr=10001, display_itr=display_itr)

## Save the trained model

In [ ]:
model.save(OUTPUT_PATH,None)
del model

# **DEFINE FUNCTIONS**

In [110]:
def eq(infilename, infilename_neg, trainfilename, gamma=1.0, K=1):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)

            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            # Calcolo il Recall a 1, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
                

        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator 

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [27]:
def aoa(infilename, infilename_neg, trainfilename, K=1):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0

        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser]))
            # Calcolo il Recall a 30, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0
            denominator += 1 

        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [125]:
def stratified(infilename, infilename_neg, trainfilename, gamma=1.0, K=30, partition=10):

    # Read pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]

    # Merge P and P_neg
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])

    Zui = dict()
    Ni = dict()

    # Compute frequencies of items in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # Compute recommendations for each user
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))

    pui = dict()
    w = dict()

    # Compute dictionary of propensity scores
    for theuser in P["users"]:
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            if theitem in pui:
                continue
            pui[theitem] = np.power(Ni[theitem], (gamma + 1) / 2.0)

    # Take the list of items (not tuples) in pui sorted by value
    items_sorted_by_value = sorted(pui, key=pui.get, reverse=True)

    # Compute linspace between the pui[0] and pui[-1] 
    linspace = np.linspace(pui[items_sorted_by_value[0]], pui[items_sorted_by_value[-1]], partition+1)
   
    # Compute dictionary w, that is, for each item, assigns the average of the puis in the partition it belongs to
    theitem = 0
    p = 0
    while theitem < len(items_sorted_by_value):  
        avg = 0
        start = theitem
        end = theitem
        # while pui[items_sorted_by_value[theitem]] > linspace[p+1]:
        while theitem < len(items_sorted_by_value) and pui[items_sorted_by_value[theitem]] >= linspace[p+1]:
            avg += 1.0 / pui[items_sorted_by_value[theitem]]
            end = theitem
            theitem += 1
        avg = avg / (end - start + 1)
        # Assign the average to the items in the partition
        for k in range(start, end+1):
            w[items_sorted_by_value[k]] = avg
        p += 1

    nonzero_user_count = 0
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    # Compute score with AUC and compute Recall
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            # Skip items with null frequency
            if  theitem not in Ni:
                continue
            # Add things to be summed for each item
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) * w[theitem]
            # Add things for recall
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 * w[theitem]
            # Increment denominator that the sum must be divided by 
            denominator += 1 / pui[theitem]

        # If there was at least one item for the user, count the user and sum the results
        if denominator > 0:
            nonzero_user_count += 1
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator 

    return {
        "auc"       : sum_user_auc / nonzero_user_count, 
        "recall"    : sum_user_recall / nonzero_user_count
    }

# **EVALUATION**

## Reproduce what was done by the authors

In [32]:
auc_results = []
recall_results = []

raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['test_data_pos_biased'] = np.load(folder_name + "biased-test_arr_pos.npy")
raw_data['test_data_neg_biased'] = np.load(folder_name + "biased-test_arr_neg.npy")
raw_data['test_data_pos_unbiased'] = np.load(folder_name + "unbiased-test_arr_pos.npy")
raw_data['test_data_neg_unbiased'] = np.load(folder_name + "unbiased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos_biased = ImplicitDataset(raw_data['test_data_pos_biased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg_biased = ImplicitDataset(raw_data['test_data_neg_biased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_pos_unbiased = ImplicitDataset(raw_data['test_data_pos_unbiased'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg_unbiased = ImplicitDataset(raw_data['test_data_neg_unbiased'], raw_data['max_user'], raw_data['max_item'])

import tensorflow as tf                 # Code to avoid tf using cached embeddings
tf.compat.v1.reset_default_graph()

model = MODEL_CLASS(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=model, sampler=sampler,
                                     eval_save_prefix=OUTPUT_PATH + DATASET_NAME,
                                     item_serving_size=500)
auc_evaluator = AUC()

model.load(OUTPUT_PATH)

model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos_biased, test_dataset_neg_biased])
model_trainer._sample_negatives(seed=10)

## Biased Evaluation

In [ ]:
model_trainer._eval_save_prefix = OUTPUT_PREFIX + "-test-pos-biased"
model_trainer._evaluate_partial(test_dataset_pos_biased)

model_trainer._eval_save_prefix = OUTPUT_PREFIX +  "-test-neg-biased"
model_trainer._evaluate_partial(test_dataset_neg_biased)

## Unbiased Evaluation

In [ ]:
model_trainer._eval_save_prefix = OUTPUT_PREFIX + "-test-pos-unbiased"
model_trainer._evaluate_partial(test_dataset_pos_unbiased)

model_trainer._eval_save_prefix = OUTPUT_PREFIX +  "-test-neg-unbiased"
model_trainer._evaluate_partial(test_dataset_neg_unbiased)

# **RESULTS**

## Compute metrics

In [126]:
biased_results = dict()

biased_results["STRATIFIED"] = stratified(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=30)
biased_results["AOA"] = aoa(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", K=30)
biased_results["UB_15"] = eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=30)
biased_results["UB_2"] =  eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=30)
biased_results["UB_25"] =  eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=30)
biased_results["UB_3"] =  eq(OUTPUT_PREFIX+"-test-pos-biased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=30)

In [127]:
unbiased_results = dict()

unbiased_results["STRATIFIED"] = stratified(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=1)
unbiased_results["AOA"] = aoa(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", K=1)
unbiased_results["UB_15"] = eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5, K=1)
unbiased_results["UB_2"] =  eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2, K=1)
unbiased_results["UB_25"] =  eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2.5, K=1)
unbiased_results["UB_3"] =  eq(OUTPUT_PREFIX+"-test-pos-unbiased_evaluate_partial.pickle", OUTPUT_PREFIX+"-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3, K=1)

In [129]:
key, value = random.choice(list(biased_results.items()))
rows = len(list(value.keys()))
columns = len(list(biased_results.items()))
results_array = np.zeros((rows,columns))

mae_results = dict()
list_biased_res = list(biased_results.keys())

for i in range(len(list_biased_res)):
    key = list_biased_res[i]

    for j in range(len(list(biased_results[key].keys()))):
        key_2 = list(biased_results[key].keys())[j]

        results_array[j][i] = abs(biased_results[key][key_2] - unbiased_results[key][key_2])

mae_df = pd.DataFrame(columns=list(biased_results.keys()), data=results_array)
metric_values = list(biased_results[list(biased_results.keys())[0]].keys())
mae_df.insert(0, "metric", metric_values)


## Show

In [133]:
mae_df.head()

,metric,STRATIFIED,AOA,UB_15,UB_2,UB_25,UB_3
0,auc,0.358019,0.151998,0.125896,0.122543,0.119977,0.118005
1,recall,0.818757,0.382071,0.265753,0.254413,0.246015,0.239736
